
{{title_its_nb2}}

{{intro}}

In this notebook, we'll read an ITS_LIVE data cube like in the previous notebook, this time focusing on strategies and approaches for working with such a large dataset *in memory*.

:::{tip} 
If you're not familiar with terms like Dask, chunking and parallelization, we *highly* suggest checking out the brief overview in [Relevant Concepts](../../background/relevant_concepts.md) and the resources linked therein.
:::
{{break}}

::::{tab-set}
:::{tab-item} Outline

(content.Section_A)=
**[A. Compare approaches for reading larger than memory data](#a-compare-approaches-for-reading-larger-than-memory-data)**
- {{a1_its_nb2}}
- {{a2_its_nb2}}
- {{a3_its_nb2}}
- {{a4_its_nb2}}

(content.Section_B)=
**[B. Organize data once its in memory](#b-organize-data-once-its-in-memory)**
- {{b1_its_nb2}}
- {{b2_its_nb2}}

::: 

:::{tab-item} Learning Goals

{{concepts}}
- Characteristics of larger than memory gridded data
- 'Lazy' v. 'non-lazy' operations

{{techniques}}
- Read + write large data with [Xarray](), [Zarr](), and [Dask]()
- Label-based indexing and selection
:::
::::


{{break}}

Expand the next cell to see specific packages used in this notebook and relevant system and version information. 

In [ ]:
%xmode minimal
import inspect
import warnings

import geopandas as gpd
import xarray as xr

warnings.simplefilter(action="ignore", category=FutureWarning)

{{break}}

## A. Compare approaches for reading larger than memory data

This section uses functions we defined in the data access notebook, all of which are stored in the `itslive_tools.py` file. If you cloned this tutorial from its github [repository](https://github.com/e-marshall/itslive) you'll see that `itslive_tools.py` is in the same directory as our current notebook, so we can import it with the following line: 

In [ ]:
import itslive_tools

Read in the catalog again, and use the `find_granule_by_point()` to find the URL that points to the ITS_LIVE granule covering your area of interest.

In [ ]:
itslive_catalog = gpd.read_file(
    "https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json"
)
# Find urls for granule covering point of interest
url = itslive_tools.find_granule_by_point([95.180191, 30.645973])
url

The function that we defined in the previous notebook, `read_in_s3()`, supports different options for reading large, chunked raster datasets. Before we use that again in this notebook, we will explore these options and the ways that they can impact how we work with the data. You can learn more about reading Zarr data with Xarray [here](https://docs.xarray.dev/en/stable/user-guide/io.html#zarr), and see the different chunking options that are supported and which we will demonstrate below [here](https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html).

### {{a1_its_nb2}}

This is the default option in `read_in_s3()`. The Xarray documentation states that `chunks='auto'` uses "dask `auto` chunking, taking into account the engine preferred chunks". 

In [ ]:
dc_auto = xr.open_dataset(url, engine="zarr", chunks="auto")
dc_auto

In this instance, the chunks of the object created with `xr.open_dataset(..., chunks='auto')` are a multiple of the on-disk chunk sizes.

In [ ]:
dc_auto

The [data model]() section/notebook discussed scalar information that is stored as attributes attached to Xarray objects. Similarly, Xarray objects read from Zarr datacubes have associated `encodings` that tell Xarray how to read and write the object to disk. We can use the encoding to learn about preferred chunking schemes. 

In [ ]:
dc_auto["v"].encoding

In [ ]:
dc_auto["v"]

For the encoding of the `v` variable, it looks like the chunking scheme is expected to be `{'mid_date': 2000, 'y':10, 'x':10}`. However, the chunks for this variable created with `chunks='auto'` are `{'mid_date': 47892, 'y': 20, 'x': 20}`. 

Let's take a look at the encoding for a 1-dimensional variable:

In [ ]:
dc_auto["vx_error"]

In [ ]:
dc_auto["vx_error"].encoding

Interesting. We see that:
- The chunk size specified in the encoding doesn't match the total length of the `mid_date` dimension. It may be an artifact from an earlier step in the data processing chain before some observations were eliminated.
- The encoding specifies a single chunk along the `mid_date` dimension for this variable, which matches the object we read into memory, the size of this chunk is just different. 

Another thing to note is that it looks like some of the variables within this `xr.Dataset` have different chunk sizes on the `y` dimension (Shown by  the error produced below). We will need to address this later before rechunking the dataset.

In [ ]:
dc_auto.chunksizes

In [ ]:
dc_auto = dc_auto.unify_chunks()

In [ ]:
dc_auto.chunksizes

### {{a2_its_nb2}}

For this argument, the documentation says: "loads the data with dask using the engine’s preferred chunk size, generally identical to the format’s chunk size. If not available, a single chunk for all arrays."

Note that with this dataset, `'auto'` and `{}` don't return the same chunking scheme.

In [ ]:
dc_set = xr.open_dataset(url, engine="zarr", chunks={})

In [ ]:
dc_set["v"].encoding

In [ ]:
dc_set["v"]

With this approach, we see that the chunking on the 3-dimensional variable we looked at above ('v') *does* match the chunking specified in the object's encoding: `{'mid_date': 20000, 'y': 10, 'x': 10}`. 

Looking at a one-dimensional variable, we see the same occurrence as with `dc_auto`: the number of chunks matches what is specified in the encoding, but the size of the chunk is different.


In [ ]:
dc_set["vx_error"].encoding

In [ ]:
dc_set["vx_error"]

The `v` and `vx_error` variables shown above have different chunk sizes along the `mid_date` dimension, so we can expect the same chunk sizes error as above, but this time for `mid_date`:

In [ ]:
dc_set.chunksizes

However this time, if try to resolve the above error like we did for `dc_auto`, We get a performance warning about the number of chunks increasing by a factor of 186.
```python
dc_set = dc_set.unify_chunks()
```

```
 PerformanceWarning: Increasing number of chunks by factor of 186_, chunked_data = chunkmanager.unify_chunks(*unify_chunks_args)
```



### {{a3_its_nb2}}

When we read this dataset from the S3 bucket, we get an object where the time dimension is not in chronological order. Because the dataset is so large, fixing this is not entirely straightforward.

Tip: It's always a good idea to look at the data!

In [ ]:
dc_set.mid_date

The standard approach would be calling Xarray's [`.sortby()`]((https://docs.xarray.dev/en/stable/generated/xarray.Dataset.sortby.html)) method:
```python
dc_set = dc_set.sortby('mid_date')
```

Performing an operation like sorting or slicing requires the entire array to be loaded into memory; for a larage dimension like `mid_date` (~48,000 elements), would be very slow and/or would max out available computationall resoures. 

There may be a chunking strategy that successfully allows one to sort this dataset along the `mid_date` dimension, but when I tried a few different re-chunking approaches, they did not work. Instead, the sucessful approach I found was a bit counterintuitive: Re-read the dataset into memory *without* dask. This let's us use Xarray's 'lazy indexing' functionality; we can sort the dataset without loading it into memory. The object will still be quite large so we will chunk the data, incorporating dask, after we sort by the time dimension.

### {{a4_its_nb2}}

We'll again use the `read_in_s3()` function, but this time passing `chunks_arg = None`. This is the same as running: `dc = xr.open_dataset(url, engine='Zarr')`. The `read_in_s3()` signature is shown below as a reminder:

In [ ]:
signature = inspect.signature(itslive_tools.read_in_s3)
print(signature)

In [ ]:
dc = itslive_tools.read_in_s3(url, chunks=None)
dc

As we saw above, the `mid_date` dimension is still out of order:

In [ ]:
dc.mid_date

## B. Organize data once it's in memory

### {{b1_its_nb2}}


But now, we can lazily perform the `.sortby()` method.

In [ ]:
dc = dc.sortby("mid_date")
dc.mid_date

Great! After some experimentation with different approaches, we have our dataset sorted in chronological order.

### {{b2_its_nb2}}

Not passing a `'chunks'` argument to `xr.open_dataset()` means that the Xarray object is a collection of Numpy arrays rather than Dask arrays. However, the dataset is still very large. We will need to use Dask even though we didn't read it in with Dask. We'll use the preferred chunking from `.encoding['chunks']` to specify a chunking scheme to the object and convert the underlying arrays from Numpy to Dask. 


However, the dataset is still very large: there are 60 variables that exist along 1,2 or, 3 dimensions (with the exception of the `mapping` variable which we will discuss later), and a single 3-d variable is 123 GB. We will still need to use Dask even though we didn't read it in as a collection of Dask arrays straight away. We will use the preferred chunk sizes we saw in the earlier objects in order to add a chunking scheme to this object and convert the numpy arrays to Dask arrays.

In [ ]:
chunking_dict = dc_auto.chunksizes

In [ ]:
dc_rechunk = dc.chunk(chunking_dict)

In [ ]:
dc_rechunk.chunks

{{conclusion}}

In this notebook, we identified a strategy for reading, chunking, and organizing this dataset that works within the memory constraints of my laptop and the size of the data. 

In the next notebook, we use vector data to narrow our focus in on a spatial area of interest and start examining ice velocity data.